In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import logging
from functools import partial

import torch.nn.functional as F
from torch import nn
import torch
import emmental
from emmental import Meta
from emmental.learner import EmmentalLearner
from emmental.model import EmmentalModel
from emmental.scorer import Scorer
from emmental.task import EmmentalTask
from modules.bert_module import BertModule
from parse_WiC import get_WiC_dataloaders
from task_config import SuperGLUE_LABEL_MAPPING, SuperGLUE_TASK_METRIC_MAPPING
from sklearn.metrics import f1_score

In [3]:
logger = logging.getLogger(__name__)

# Initalize Emmental

In [4]:
emmental.init(
    "logs",
    config={
        "model_config": {"device": 0, "dataparallel": False},
        "learner_config": {
            "n_epochs": 4,
            "valid_split": "val",
            "optimizer_config": {"optimizer": "adam", "lr": 1e-5},
            "min_lr": 0,
            "lr_scheduler_config": {
                "warmup_percentage": 0.1,
                "lr_scheduler": None,
            },
        },
        "logging_config": {
            "counter_unit": "batch",
            "evaluation_freq": 100,
            "checkpointing": True,
            "checkpointer_config": {
                "checkpoint_metric": {"WiC/SuperGLUE/val/accuracy":"max"},
                "checkpoint_freq": 1,
            },
        },
    },
)

[2019-05-30 16:11:56,143][INFO] emmental.meta:95 - Setting logging directory to: logs/2019_05_30/16_11_56
[2019-05-30 16:11:56,154][INFO] emmental.meta:56 - Loading Emmental default config from /dfs/scratch0/bradenjh/emmental/src/emmental/emmental-default-config.yaml.
[2019-05-30 16:11:56,154][INFO] emmental.meta:143 - Updating Emmental config from user provided config.


In [5]:
Meta.config

{'meta_config': {'seed': 0, 'verbose': True, 'log_path': None},
 'model_config': {'model_path': None, 'device': 0, 'dataparallel': False},
 'learner_config': {'fp16': False,
  'n_epochs': 4,
  'train_split': 'train',
  'valid_split': 'val',
  'test_split': 'test',
  'ignore_index': -100,
  'optimizer_config': {'optimizer': 'adam',
   'lr': 1e-05,
   'l2': 0.0,
   'grad_clip': 1.0,
   'sgd_config': {'momentum': 0.9},
   'adam_config': {'betas': (0.9, 0.999)}},
  'lr_scheduler_config': {'lr_scheduler': None,
   'warmup_steps': None,
   'warmup_unit': 'batch',
   'warmup_percentage': 0.1,
   'min_lr': 0.0,
   'linear_config': {'min_lr': 0.0},
   'exponential_config': {'gamma': 0.9},
   'plateau_config': {'factor': 0.5, 'patience': 10, 'threshold': 0.0001}},
  'task_scheduler': 'round_robin',
  'global_evaluation_metric_dict': None,
  'min_lr': 0},
 'logging_config': {'counter_unit': 'batch',
  'evaluation_freq': 100,
  'writer_config': {'writer': 'tensorboard', 'verbose': True},
  'checkp

In [6]:
import os

TASK_NAME = "WiC"
DATA_DIR = "/dfs/scratch0/bradenjh/superglue" #os.environ["SUPERGLUEDATA"]
BERT_MODEL_NAME = "bert-large-cased"
BATCH_SIZE = 4

# Extract train/dev dataset from file

In [7]:
dataloaders = get_WiC_dataloaders(
    data_dir=DATA_DIR,
    task_name=TASK_NAME,
    splits=["train", "val", "test"],
    max_sequence_length=128,
    max_data_samples=None,
    tokenizer_name=BERT_MODEL_NAME,
    batch_size=BATCH_SIZE,
)

[2019-05-30 16:11:56,322][INFO] tokenizer:8 - Loading Tokenizer bert-large-cased
[2019-05-30 16:11:56,588][INFO] pytorch_pretrained_bert.tokenization:190 - loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-vocab.txt from cache at /afs/cs.stanford.edu/u/bradenjh/.pytorch_pretrained_bert/cee054f6aafe5e2cf816d2228704e326446785f940f5451a5b26033516a4ac3d.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


/dfs/scratch0/bradenjh/superglue/WiC/train.jsonl
{'label': False, 'word': 'carry', 'pos': 'V', 'sentence1': 'You must carry your camping gear .', 'sentence2': 'Sound carries well over water .', 'sentence1_idx': '2', 'sentence2_idx': '1', 'idx': 0}


[2019-05-30 16:11:58,519][INFO] parse_WiC:154 - Loaded train for WiC.


max len 68
/dfs/scratch0/bradenjh/superglue/WiC/val.jsonl
{'label': False, 'word': 'board', 'pos': 'N', 'sentence1': 'Room and board .', 'sentence2': 'He nailed boards across the windows .', 'sentence1_idx': '2', 'sentence2_idx': '2', 'idx': 0}


[2019-05-30 16:11:58,826][INFO] parse_WiC:154 - Loaded val for WiC.


max len 57
/dfs/scratch0/bradenjh/superglue/WiC/test.jsonl
{'word': 'defeat', 'pos': 'N', 'sentence1': 'It was a narrow defeat .', 'sentence2': "The army 's only defeat .", 'sentence1_idx': '4', 'sentence2_idx': '4', 'idx': 0}


[2019-05-30 16:11:59,354][INFO] parse_WiC:154 - Loaded test for WiC.


max len 60


# Build Emmental task

In [8]:
def ce_loss(task_name, immediate_ouput_dict, Y, active):
    module_name = f"{task_name}_pred_head"
    return F.cross_entropy(
        immediate_ouput_dict[module_name][0][active], (Y.view(-1) - 1)[active]
    )

In [9]:
def output(task_name, immediate_ouput_dict):
    module_name = f"{task_name}_pred_head"
    return F.softmax(immediate_ouput_dict[module_name][0], dim=1)

In [10]:
def macro_f1(golds, probs, preds):
    return {"macro_f1": f1_score(golds, preds, average="macro")}

In [11]:
class LinearModule(nn.Module):
    def __init__(self, feature_dim, class_cardinality):
        super().__init__()

        self.linear = nn.Linear(feature_dim, class_cardinality)

    def forward(self, feature, idx1, idx2):
        last_layer = feature[-1]
        emb = last_layer[:,0,:]
        idx1 = idx1.unsqueeze(-1).unsqueeze(-1).expand([-1, -1, last_layer.size(-1)])
        idx2 = idx2.unsqueeze(-1).unsqueeze(-1).expand([-1, -1, last_layer.size(-1)])
        word1_emb = last_layer.gather(dim=1, index=idx1).squeeze(dim=1)
        word2_emb = last_layer.gather(dim=1, index=idx2).squeeze(dim=1)
        input = torch.cat([emb, word1_emb, word2_emb], dim=-1)
        return self.linear.forward(input)

In [12]:
BERT_OUTPUT_DIM = 768 if "base" in BERT_MODEL_NAME else 1024
TASK_CARDINALITY = (
    len(SuperGLUE_LABEL_MAPPING[TASK_NAME].keys())
    if SuperGLUE_LABEL_MAPPING[TASK_NAME] is not None
    else 1
)

emmental_task = EmmentalTask(
    name=TASK_NAME,
    module_pool=nn.ModuleDict(
        {
            "bert_module": BertModule(BERT_MODEL_NAME),
            f"{TASK_NAME}_pred_head": LinearModule(3 * BERT_OUTPUT_DIM, TASK_CARDINALITY),
        }
    ),
    task_flow=[
        {
            "name": "input",
            "module": "bert_module",
            "inputs": [("_input_", "token_ids"), ("_input_", "token_segments")],
        },
        {
            "name": f"{TASK_NAME}_pred_head",
            "module": f"{TASK_NAME}_pred_head",
            "inputs": [("input", 0), ("_input_", "sent1_idxs"), ("_input_", "sent2_idxs")],
        },
    ],
    loss_func=partial(ce_loss, TASK_NAME),
    output_func=partial(output, TASK_NAME),
    scorer=Scorer(
        metrics=SuperGLUE_TASK_METRIC_MAPPING[TASK_NAME]
    ),
)

[2019-05-30 16:11:59,844][INFO] pytorch_pretrained_bert.modeling:580 - loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased.tar.gz from cache at ./cache/7fb0534b83c42daee7d3ddb0ebaa81387925b71665d6ea195c5447f1077454cd.eea60d9ebb03c75bb36302aa9d241d3b7a04bba39c360cf035e8bf8140816233
[2019-05-30 16:11:59,845][INFO] pytorch_pretrained_bert.modeling:588 - extracting archive file ./cache/7fb0534b83c42daee7d3ddb0ebaa81387925b71665d6ea195c5447f1077454cd.eea60d9ebb03c75bb36302aa9d241d3b7a04bba39c360cf035e8bf8140816233 to temp dir /tmp/tmpm0qfx241
[2019-05-30 16:12:12,072][INFO] pytorch_pretrained_bert.modeling:598 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pooler_fc_size": 768,
  "pooler_num_a

In [13]:
mtl_model = EmmentalModel(name="SuperGLUE_single_task", tasks=[emmental_task])

[2019-05-30 16:12:19,695][INFO] emmental.model:58 - Moving model to GPU (cuda:0).
[2019-05-30 16:12:23,153][INFO] emmental.model:44 - Created emmental model SuperGLUE_single_task that contains task {'WiC'}.
[2019-05-30 16:12:23,154][INFO] emmental.model:58 - Moving model to GPU (cuda:0).


In [14]:
emmental_learner = EmmentalLearner()

In [15]:
# emmental_learner.learn(mtl_model, dataloaders.values())

In [16]:
mtl_model.score(dataloaders["val"])

{'WiC/SuperGLUE/val/accuracy': 0.5047021943573667}

In [17]:
# mtl_model.score(dataloaders["train"])

{'WiC/SuperGLUE/train/accuracy': 0.502210759027266}

In [18]:
# PKL_PATH = "/dfs/scratch0/bradenjh/emmental-tutorials/superglue/models/WiC_BERT2"
PKL_PATH = "/dfs/scratch0/bradenjh/emmental-tutorials/superglue/logs/2019_05_29/13_59_55/best_model_WiC_SuperGLUE_val_accuracy.pth"

In [19]:
# mtl_model.save(PKL_PATH)

In [20]:
new_model = EmmentalModel(name="SuperGLUE_single_task", tasks=[emmental_task])
new_model.load(PKL_PATH)
new_model.score(dataloaders["val"])

[2019-05-30 16:14:10,313][INFO] emmental.model:58 - Moving model to GPU (cuda:0).
[2019-05-30 16:14:10,318][INFO] emmental.model:44 - Created emmental model SuperGLUE_single_task that contains task {'WiC'}.
[2019-05-30 16:14:10,319][INFO] emmental.model:58 - Moving model to GPU (cuda:0).
/dfs/scratch0/bradenjh/emmental-tutorials/.venv/lib/python3.6/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'modules.bert_module.BertModule' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
[2019-05-30 16:14:23,070][INFO] emmental.model:412 - [SuperGLUE_single_task] Model loaded from /dfs/scratch0/bradenjh/emmental-tutorials/superglue/logs/2019_05_29/13_59_55/best_model_WiC_SuperGLUE_val_accuracy.pth
[2019-05-30 16:14:23,072][INFO] emmental.model:58 - Moving model to GPU (cuda:0).


{'WiC/SuperGLUE/val/accuracy': 0.731974921630094}

In [55]:
import json

from task_config import (
    SuperGLUE_LABEL_MAPPING, 
    SuperGLUE_TASK_METRIC_MAPPING, 
    SuperGLUE_TASK_SPLIT_MAPPING
)

SPLIT = "val"

def make_analysis_df(model):
    # Get predictions
    gold_dict, prob_dict, pred_dict = model.predict(dataloaders[SPLIT], return_preds=True)
    probs = prob_dict["WiC"][:,0]
    preds = pred_dict["WiC"]

    # Load raw data
    jsonl_path = os.path.join(
        DATA_DIR, TASK_NAME, SuperGLUE_TASK_SPLIT_MAPPING[TASK_NAME][SPLIT]
    )

    # Add new columns
    rows = [json.loads(row) for row in open(jsonl_path, encoding="utf-8")]
    for i, row in enumerate(rows):
        row["prob"] = probs[i]
        row["pred"] = True if preds[i] == 1 else False
        row["correct"] = "Y" if row["pred"] == row["label"] else "N"

    # Make tsv
    df = pd.DataFrame(rows)
    df = df[['idx', 'label', 'pred', 'prob', 'correct', 'word', 'pos', 'sentence1', 'sentence2']]
    return df

df = make_analysis_df(new_model)

In [56]:
TUTORIALS_ROOT = "/dfs/scratch0/bradenjh/emmental-tutorials/"

out_path = os.path.join(TUTORIALS_ROOT, "superglue", "analysis", f"WiC_{SPLIT}_analysis_v0.csv")
df.to_csv(out_path)
print(f"Wrote error analysis to {out_path}")

Wrote error analysis to /dfs/scratch0/bradenjh/emmental-tutorials/superglue/analysis/WiC_val_analysis_v0.csv


In [57]:
df.head(1)

,idx,label,pred,prob,correct,word,pos,sentence1,sentence2
0,0,False,False,0.001316,Y,board,N,Room and board .,He nailed boards across the windows .


In [68]:
import nltk
from nltk.corpus import stopwords as nltk_stopwords
stopwords = set(nltk_stopwords.words('english'))

preds = []
labels = []

def get_ngrams(tokens, window=1):
    num_ngrams = len(tokens) - window + 1
    for i in range(num_ngrams):
        yield tokens[i:i+window]
        
for index, row in df.iterrows():
    word = row["word"]
    labels.append(1 if row["label"] == True else 2)
    trigrams = []
    for sent in ["sentence1", "sentence2"]:
        tokens = row[sent].split()
#         if sum([word in tok for tok in tokens]) > 1:
#             print(row[sent])
        for i, tok in enumerate(tokens):
            if word in tok:
                idx = i
                break
        trigrams.append([' '.join(ngram) 
                         for ngram in get_ngrams(tokens[idx-2:idx+2], window=3) 
                         if len(ngram) == 3])
    if (set(trigrams[0]).intersection(set(trigrams[1]))):
        preds.append(1)
        print(trigrams)
        print(f"{word}: {row['sentence1']}.....{row['sentence2']}")
        print()        
    else:
        preds.append(0)


[['on the bridge', 'the bridge of'], ['break the bridge', 'the bridge of']]
bridge: Her glasses left marks on the bridge of her nose ......Rugby players often break the bridge of their noses .

[['at the finish', 'the finish .'], ['at the finish', 'the finish .']]
finish: My horse was several lengths behind at the finish ......The winner is the team with the most points at the finish .

[['Sculpture in contradistinction', 'in contradistinction to'], ['soda in contradistinction', 'in contradistinction to']]
contradistinction: Sculpture in contradistinction to painting ......We used hamburgers and soda in contradistinction to healthy food .

[['felt a stream', 'a stream of'], ['ejected a stream', 'a stream of']]
stream: He felt a stream of air ......The hose ejected a stream of water .

[['is an obstacle', 'an obstacle to'], ['is an obstacle', 'an obstacle to']]
obstacle: Lack of imagination is an obstacle to one 's advancement ......The poverty of a district is an obstacle to good educa

In [63]:
from metal.metrics import *

print(accuracy_score(labels, preds, ignore_in_pred=[0]))
print(coverage_score(labels, preds))

0.75
0.025078369905956112
